In [12]:
import numpy as np
import astropy.units as u

from gw_signal_tools.matrix_class import MatrixWithUnit

In [2]:
test = MatrixWithUnit([2, 2], [u.s, u.m])

In [3]:
float(test)

TypeError: only length-1 arrays can be converted to Python scalars

In [5]:
float(np.array([1.0], dtype=object))
# float(np.array([1.0 * u.s], dtype=object))

/tmp/ipykernel_366939/3182476769.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  float(np.array([1.0], dtype=object))


1.0

In [16]:
class Fisher:
    def __init__(self, with_units=True):
        self._fisher = 42
        self._with_units = with_units
    
    # def fisher(self, with_units=False):
    #     if with_units:
    #         return self._fisher * u.s
    #     else:
    #         return self._fisher
    
    # @property
    # def fisher(self):
    #     self.fisher(self._with_units)
    

    # @property is cool because no fisher(...) needed and setters/getters
    # are custom. On the other hand, units thing is not convenient...
    # -> maybe make separate properties? One fisher and one fisher_with_units?
    @property
    def fisher(self):
        if self._with_units:
            return self._fisher
        else:
            return 96

    @property
    def return_with_units(self):
        return self._with_units
    
    @return_with_units.setter
    def return_with_units(self, value):
        if value != True and value != False:
            raise ValueError()
        else:
            self._with_units = value

In [6]:
from gwpy.frequencyseries import FrequencySeries

test = FrequencySeries([1])

list(test.xspan)

[0.0, 1.0]

In [18]:
test = Fisher()

print(test.fisher)

test._with_units = False

print(test.fisher)


42
96


Nice Idea: have value and unit properties

In [ ]:
class FisherMatrix:
    def __new__(cls):
        ...

        # return self._value * self._unit


    @property
    def value(self):
        ...
        # return self._value
    

    @property
    def unit(self):
        ...
        # return self._unit

    @classmethod
    def plot(self):
        ...
        # plot with color=uncertainty


Hmmm, inverse as property is not convenient... Maybe make base class MatrixWithUnit and then have Fisher + its inverse being instances of that?

In [555]:
# class MatrixWithUnit(np.ndarray):
#     def __new__(cls, value, **kwargs):
#         new.value = super().__new__(np.shape(value), dtype=float, **kwargs)

#         new.unit = super().__new__(np.shape(value), dtype=object)
        # return self._value * self._unit



class MatrixWithUnit:
    _allowed_numeric_types = (int, float, complex, np.number)
    _allowed_unit_types = (u.IrreducibleUnit, u.CompositeUnit, u.Unit)


    def  __init__(self, value, unit) -> None:
        assert np.shape(value) == np.shape(unit), \
                ('`value` and `unit` must have equal shape if unit is an '
                'array of astropy units.')
        

        # Idea: add support for given unit that is "scalar"
        # -> should we also make sure value is np.array?

        # value = np.asarray(value, dtype=np.number)
        # value = np.asarray(value, dtype=float)

        # if isinstance(unit, self._allowed_unit_types):
        #     if not isinstance(value, self._allowed_numeric_types):
        #         unit = np.full(np.shape(value), unit, dtype=object)
        # else:
        #     assert np.shape(value) == np.shape(unit), \
        #         ('`value` and `unit` must have equal shape if unit is an '
        #         'array of astropy units.')
            

        self.value = value
        self.unit = unit
        # NOTE: setting "private" properties here already is not good practice.
        # Instead go through setters of attributes, where these are set
        

    # def __new__(cls, value, unit):
    #     cls._value = value
    #     cls._unit = unit

    #     """
    #     TODOS
    #     - make sure they have same shape
    #     - maybe make sure unit has dtype object? Or that all members are of type u.Quantity?
    #     """
        
    #     return cls._value * cls._unit


    @property
    def value(self):
        ...
        return self._value
    
    @value.setter
    def value(self, value):
        try:
            assert np.shape(value) == np.shape(self.value), \
                'New and old `value` must have equal shape'
        except AttributeError:
            pass  # New class instance is created, nothing to check
            
        for val in np.reshape(value, -1):
            assert (isinstance(val, self._allowed_numeric_types)
                    and not isinstance(val, bool)), \
                f'Need valid numeric types for all members of `value` (not {type(val)}).'
        
        self._value = value


    @property
    def unit(self):
        ...
        return self._unit
    
    @unit.setter
    def unit(self, unit):
        try:
            assert np.shape(unit) == np.shape(self.unit), \
                'New and old `unit` must have equal shape'
        except AttributeError:
            pass  # New class instance is created, nothing to check
        
        for val in np.reshape(unit, -1):
            assert isinstance(val, self._allowed_unit_types), \
                f'Need valid unit types for all members of `unit` (not {type(val)}).'
        
        self._unit = unit


    def __repr__(self) -> str:
        return (self.value * self.unit).__repr__()
    

    def __float__(self):
        # if np.shape(self.value)
        return float(self.value)
    

    def __copy__(self):
        return MatrixWithUnit(self.value.__copy__(), self.unit.__copy__())
    
    def copy(self):
        return self.__copy__()
    

    def __eq__(self, other):
        return self.value.__eq__(self.other.value) and self.unit.__eq__(self.other.unit)
    

    def __hash__(self):
        return hash(self.value) ^ hash(self.unit)
    

    def __getitem__(self, key):
        return self.value.__getitem__(key) * self.unit.__getitem__(key)
    

    def __setitem__(self, key, value):
        try:
            self.value.__setitem__(key, value.value)
            self.unit.__setitem__(key, value.unit)
        except AttributeError:
            # if isinstance(value, u.core.IrreducibleUnit):
            #     # raise TypeError(
            #     #     'Cannot set just unit.'
            #     # )
            #     self.unit.__setitem__(key, value)
            # else:
            #     self.value.__setitem__(key, value)

            # Hmmm, setting only value or unit should not be allowed, right?

            try:
                value = u.Quantity(value)
            except TypeError:
                raise TypeError(
                    'Can only set items to data types that have members'
                    '`value` and `unit` (such as astropy Quantities or '
                    'MatrixWithUnits) or can be converted into a Quantity.'
                )
            
            self.value.__setitem__(key, value.value)
            self.unit.__setitem__(key, value.unit)
    

    # Add properties for add, sub, mult, truediv, rtruediv; matmul maybe as well?
    def __add__(self, other):
        if isinstance(other, self._allowed_numeric_types):
            return MatrixWithUnit(self.value + other, self.unit)
        # elif isinstance(other, self._allowed_unit_types):
        #     return MatrixWithUnit(self.value, self.unit + np.asarray(other, dtype=object))
        # Adding just units without values does not make much sense, right?
        elif isinstance(other, u.Quantity):
            assert np.all(other.unit == self.unit)

            return MatrixWithUnit(self.value + other.value, self.unit)
        elif isinstance(other, MatrixWithUnit):
            assert np.all(other.unit == self.unit)
            
            return MatrixWithUnit(self.value + other.value, self.unit)
        else:
            try:
                return MatrixWithUnit(self.value * other, self.unit)
            except:
                raise TypeError(
                    f'Addition between {type(other)} and `MatrixWithUnit`'
                    ' is not supported.'
                )
    

    def __radd__(self, other):
        return self.__add__(other)
            

    def __mul__(self, other):
        if isinstance(other, self._allowed_numeric_types):
            return MatrixWithUnit(self.value * other, self.unit)
        elif isinstance(other, self._allowed_unit_types):
            return MatrixWithUnit(self.value, self.unit * np.asarray(other, dtype=object))
        elif isinstance(other, u.Quantity):
            return MatrixWithUnit(self.value * other.value, self.unit * other.unit)
        elif isinstance(other, MatrixWithUnit):
            return MatrixWithUnit(self.value * other.value, self.unit * other.unit)
        else:
            try:
                return MatrixWithUnit(self.value * other, self.unit)
            except:
                raise TypeError(
                    f'Multiplication between {type(other)} and `MatrixWithUnit`'
                    ' is not supported.'
                )
    

    def __rmul__(self, other):
        return self.__mul__(other)


    def __truediv__(self, other):
        if isinstance(other, self._allowed_numeric_types):
            return MatrixWithUnit(self.value / other, self.unit)
        elif isinstance(other, self._allowed_unit_types):
            return MatrixWithUnit(self.value, self.unit / np.asarray(other, dtype=object))
        elif isinstance(other, u.Quantity):
            return MatrixWithUnit(self.value / other.value, self.unit / other.unit)
        elif isinstance(other, MatrixWithUnit):
            return MatrixWithUnit(self.value / other.value, self.unit / other.unit)
        else:
            try:
                return MatrixWithUnit(self.value / other, self.unit)
            except:
                raise TypeError(
                    f'Division of `MatrixWithUnit` and {type(other)}'
                    ' is not supported.'
                )
    

    def __rtruediv__(self, other):
        # Important: 1/u.Unit becomes quantity, we have to use power -1 so that
        # unit actually remains a unit
        if isinstance(other, self._allowed_numeric_types):
            return MatrixWithUnit(other / self.value, self.unit**-1)
        elif isinstance(other, self._allowed_unit_types):
            return MatrixWithUnit(1.0 / self.value, np.asarray(other, dtype=object) / self.unit)
        elif isinstance(other, u.Quantity):
            return MatrixWithUnit(other.value / self.value, other.unit / self.unit)
        elif isinstance(other, MatrixWithUnit):
            return MatrixWithUnit(other.value / self.value, other.unit / self.unit)
        else:
            try:
                return MatrixWithUnit(other / self.value, self.unit**-1)
            except:
                raise TypeError(
                    f'Division of {type(other)} and `MatrixWithUnit`'
                    ' is not supported.'
                )
            
    def __pow__(self, other):
        if isinstance(other, self._allowed_numeric_types):
            return MatrixWithUnit(self.value.__pow__(other), self.unit.__pow__(other))
        else:
            raise TypeError(
                'Raising of `MatrixWithUnit` to a non-numeric type like '
                f'{type(other)} is not supported.'
            )

    # Do we have to make manual slicing?

In [556]:
isinstance(u.m, u.IrreducibleUnit)

True

In [557]:
type(u.m**-1)

astropy.units.core.CompositeUnit

In [558]:
for val in np.reshape(42, -1):
    print(val)
    print(type(val))
    print(isinstance(val, (int, float, complex, np.number)))

42
<class 'numpy.int64'>
True


In [559]:
isinstance(2.0 * u.m, u.Quantity)

# u.Quantity(u.m)
type(u.m)


astropy.units.core.IrreducibleUnit

In [560]:
test = MatrixWithUnit(42, u.s)

print(test)
print(type(test))
print(test.value)
print(test.unit)

<Quantity 42. s>
<class '__main__.MatrixWithUnit'>
42.0
s


In [561]:
np.shape(42)

()

In [562]:
test_mat = np.ones((2, 2))
test_unit = np.array([[u.s, u.m], [u.m, u.kg]])

test_mat_unit = MatrixWithUnit(test_mat, test_unit)

In [563]:
test_mat_unit * test_mat_unit
test_mat_unit * 2.0
2.0 * test_mat_unit

array([[<Quantity 2. s>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 2. kg>]], dtype=object)

In [564]:
1.0/test_unit

array([[<Quantity 1. 1 / s>, <Quantity 1. 1 / m>],
       [<Quantity 1. 1 / m>, <Quantity 1. 1 / kg>]],
      dtype=object)

In [565]:
mult_test_1 = test_mat_unit * 2.0
mult_test_2 = test_mat_unit * u.m
# mult_test_3 = test_unit * u.m
mult_test_4 = np.asarray(u.m, dtype=object) * test_unit
mult_test_5 = test_mat_unit * test_mat_unit
mult_test_6 = 1.0 / test_mat_unit
mult_test_7 = test_mat_unit / 1.0
mult_test_8 = test_mat_unit / test_mat_unit
mult_test_9 = test_mat_unit**2

print(mult_test_1)
print(mult_test_1.value)
print(mult_test_1.unit)

print(mult_test_2)
print(mult_test_2.value)
print(mult_test_2.unit)
# print(mult_test_3)
print(mult_test_4)
print(mult_test_5)
print(mult_test_6)
print(mult_test_7)
print(mult_test_8)
print(mult_test_9)

array([[<Quantity 2. s>, <Quantity 2. m>],
       [<Quantity 2. m>, <Quantity 2. kg>]], dtype=object)
[[2. 2.]
 [2. 2.]]
[[Unit("s") Unit("m")]
 [Unit("m") Unit("kg")]]
array([[<Quantity 1. m s>, <Quantity 1. m2>],
       [<Quantity 1. m2>, <Quantity 1. kg m>]], dtype=object)
[[1. 1.]
 [1. 1.]]
[[Unit("m s") Unit("m2")]
 [Unit("m2") Unit("kg m")]]
[[Unit("m s") Unit("m2")]
 [Unit("m2") Unit("kg m")]]
array([[<Quantity 1. s2>, <Quantity 1. m2>],
       [<Quantity 1. m2>, <Quantity 1. kg2>]], dtype=object)
array([[<Quantity 1. 1 / s>, <Quantity 1. 1 / m>],
       [<Quantity 1. 1 / m>, <Quantity 1. 1 / kg>]],
      dtype=object)
array([[<Quantity 1. s>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 1. kg>]], dtype=object)
array([[<Quantity 1.>, <Quantity 1.>],
       [<Quantity 1.>, <Quantity 1.>]], dtype=object)
array([[<Quantity 1. s2>, <Quantity 1. m2>],
       [<Quantity 1. m2>, <Quantity 1. kg2>]], dtype=object)


In [566]:
test_mat_unit = MatrixWithUnit(test_mat, test_unit)
print(test_mat_unit)

print(test_mat_unit[0, 0])  # Test getter
test_mat_unit[0, 0] = 42  # Test setter

print(test_mat_unit)

test_mat_unit[1, 1] = 96 * u.pc  # Another setter test

print(test_mat_unit)


# test_mat_unit[0, 0] = u.m  # Error, good

# print(test_mat_unit)

array([[<Quantity 1. s>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 1. kg>]], dtype=object)
1.0 s
array([[<Quantity 42.>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 1. kg>]], dtype=object)
array([[<Quantity 42.>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 96. pc>]], dtype=object)


In [567]:
test = MatrixWithUnit(42, u.s)
print(type(test))

test.value = 96

print(test)

test.unit = u.m

print(test)


print(float(test))

# print(float(test_mat_unit))

<class '__main__.MatrixWithUnit'>
<Quantity 96. s>
<Quantity 96. m>
96.0


In [568]:
test_arr = np.array([[u.s, u.m]], dtype=u.core.IrreducibleUnit)

test_arr[0, 0]

Unit("s")

In [569]:
test_copy = test_mat_unit.copy()

test_copy[0, 0] = -1 * u.kg

print(test_copy)
print(test_mat_unit)  # Unchanged, good. If we set equal without copy, then it also changes, so things seem to work

array([[<Quantity -1. kg>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 96. pc>]], dtype=object)
array([[<Quantity 42.>, <Quantity 1. m>],
       [<Quantity 1. m>, <Quantity 96. pc>]], dtype=object)


In [570]:
test_addition_1 = test_mat_unit + 1.0
test_addition_2 = 1.0 + test_mat_unit
# test_addition_3 = test_mat_unit + u.m  # Should throw error, as it does
# test_addition_4 = test_mat_unit + test_mat_unit.unit  # Should throw error
test_addition_5 = test_mat_unit + test_mat_unit

# test = MatrixWithUnit(42, u.m)
test = MatrixWithUnit(np.array([[42, 96]]), u.m)
test_addition_6 = 2.0 * u.m + test
test_addition_7 = test + 2.0 * u.m
# test_addition_8 = test + 2.0 * u.s  # Incompatible unit, should throw error, as it does

print(test_addition_1)
print(test_addition_2)
print(test_addition_5)
print(test_addition_6)
print(test_addition_7)

AttributeError: 'numpy.ndarray' object has no attribute '_get_converter'

In [572]:
test = MatrixWithUnit([[42, 96]], u.m)

print(test)

array([[<Quantity 42. m>, <Quantity 96. m>]], dtype=object)
